In [8]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [9]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
method = METHOD_SCALAR_QUANTIZE
nbits = 8

In [10]:
linear_name = 'fc1'
feedback_bits = 256
ncodebooks=1024
ncentroids=16
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [11]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 

In [12]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)

running method:  ScalarQuantize


In [13]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [15]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, -1, y_out_last.shape[-1])
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last)

[[-9.26360826e-02  2.73183779e-02 -5.36655769e-01 ... -6.51589815e-02
   8.91088498e-02  1.77712033e-03]
 [-2.30495191e-01 -1.20754990e-01 -2.99116508e-01 ... -3.72805448e-01
   2.17306111e-02 -7.54524877e-02]
 [ 3.92938651e-01 -5.56508640e-02 -2.27052997e-01 ... -4.89370099e-01
   1.90694682e-01  1.20972548e-01]
 ...
 [ 3.40133408e-01  1.22394998e-01 -5.21858451e-01 ... -1.70391861e-01
  -2.04582131e-01  2.06138395e-03]
 [ 2.34704078e-01  6.06599665e-03 -2.43795041e-01 ...  6.40264021e-02
  -2.90434418e-04  4.52755897e-02]
 [-3.04111327e-01  1.16784648e-01 -4.31115679e-01 ... -3.18567059e-01
  -1.50198575e-01  1.83542177e-01]]
y_out_last.shape:  (32000, 256)
y_out_last_re.shape:  (1000, 32, 256)
